# Peace-Sign Selfie (Colab-compatible)

This notebook opens the webcam (in Colab or a local Jupyter runtime), uses Mediapipe to detect a ✌️ peace sign (index + middle fingers up), shows a 2-second countdown, takes a selfie, saves it as `selfie_YYYYMMDD_HHMMSS.jpg`, applies a simple filter, and displays the result.

Requirements: Python, OpenCV, Mediapipe. Install commands are below. If running in Colab, you may need to accept camera permissions in the browser. If you run this in a local Jupyter (not Colab), the normal OpenCV camera code will be used instead.

In [ ]:
# Install dependencies (run this cell first)
!pip install --quiet mediapipe opencv-python-headless==4.7.0.72 numpy
# opencv-python-headless is used in Colab; if running locally and you need GUI windows, install opencv-python instead:
# !pip install opencv-python mediapipe numpy

In [ ]:
# Imports and helper functions
import cv2
import mediapipe as mp
import numpy as np
import time
from datetime import datetime
import os
from base64 import b64decode

# Colab-specific helpers (use eval_js to access browser camera stream)
try:
    from google.colab.output import eval_js
    from google.colab.patches import cv2_imshow
    IN_COLAB = True
except Exception:
    IN_COLAB = False

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Finger landmark indexes (Mediapipe)
TIP_IDS = [4, 8, 12, 16, 20]  # thumb, index, middle, ring, pinky

def is_finger_up(hand_landmarks, finger_tip_idx, finger_pip_idx):
    # hand_landmarks: mediapipe normalized landmarks list; returns True if finger appears up.
    # In image coords, y increases downwards. A finger is 
 if tip y < pip y.
    return hand_landmarks.landmark[finger_tip_idx].y < hand_landmarks.landmark[finger_pip_idx].y

def detect_peace_sign(hand_landmarks):
    # Returns True if index and middle fingers are up, ring and pinky are down. Thumb ignored for simplicity.
    # finger tip and pip indices for mediapipe: index tip=8, pip=6; middle tip=12, pip=10; ring tip=16, pip=14; pinky tip=20, pip=18
    try:
        idx_up = is_finger_up(hand_landmarks, 8, 6)
        mid_up = is_finger_up(hand_landmarks, 12, 10)
        ring_up = is_finger_up(hand_landmarks, 16, 14)
        pinky_up = is_finger_up(hand_landmarks, 20, 18)
        # Peace sign: index & middle up; ring & pinky down.
        return idx_up and mid_up and (not ring_up) and (not pinky_up)
    except Exception:
        return False

def apply_filter(img, mode='sepia'):
    # img in BGR (OpenCV)
    if mode == 'grayscale':
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    elif mode == 'sepia':
        # simple sepia kernel
        kernel = np.array([[0.272, 0.534, 0.131],
                           [0.349, 0.686, 0.168],
                           [0.393, 0.769, 0.189]])
        img_sepia = cv2.transform(img, kernel)
        img_sepia = np.clip(img_sepia, 0, 255).astype(np.uint8)
        return img_sepia
    elif mode == 'cartoon':
        # cartoon effect: bilateral filter + edge mask
        img_color = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.medianBlur(img_gray, 7)
        edges = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        cartoon = cv2.bitwise_and(img_color, edges_colored)
        return cartoon
    else:
        return img

def timestamp_filename(basename='selfie', ext='jpg'):
    t = datetime.now().strftime('%Y%m%d_%H%M%S')
    return f"{basename}_{t}.{ext}"

## Colab camera helper (if running in Google Colab)

The following starts the browser camera once and lets Python fetch frames repeatedly using `get_frame()`. If you're running this notebook in a local Jupyter environment with direct OpenCV access, you can skip this cell and use the local camera code in the main loop instead.

In [ ]:
if IN_COLAB:
    print('Running in Colab environment. Using browser camera streaming helper.')
    def start_camera(width=640, height=480):
        # Initializes browser camera and stores references to a video element and canvas in window._video/_canvas
        js = f"
        async function startCamera(){
          const video = document.createElement('video');
          video.style.display = 'block';
          video.width = {width};
          video.height = {height};
          document.body.appendChild(video);
          const stream = await navigator.mediaDevices.getUserMedia({ video: true });
          video.srcObject = stream;
          await video.play();
          const canvas = document.createElement('canvas');
          canvas.width = {width};
          canvas.height = {height};
          window._video = video;
          window._canvas = canvas;
          return true;
        }
        startCamera();
        "
        return eval_js(js)

    def get_frame():
        # Returns an OpenCV BGR image captured from the browser video element.
        data = eval_js(

,
,

)
        header, encoded = data.split(',', 1)
        img_bytes = b64decode(encoded)
        nparr = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        return img
else:
    print('Not running in Colab. Will try to use local camera via OpenCV VideoCapture.')

# End of camera helper cell

## Main loop: detect gesture and take selfie

Run this cell to start the live detection. In Colab, first run the start_camera cell and allow camera permissions. Then run this cell. Use Ctrl+C in a local runtime to stop, or interrupt the kernel in Colab.

In [ ]:
# Main loop implementation
SAVE_DIR = '/content' if IN_COLAB else os.getcwd()
os.makedirs(SAVE_DIR, exist_ok=True)
FILTER = 'sepia'  # choose 'grayscale', 'sepia', or 'cartoon'
DELAY_SECONDS = 2  # delay before taking selfie after gesture detected

if IN_COLAB:
    start_camera(640, 480)
    time.sleep(1.0)

cap = None
if not IN_COLAB:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError('Could not open camera. Is camera available?')

hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.6, min_tracking_confidence=0.5)
gesture_active = False
gesture_start = None
captured = False
last_frame = None

print('Starting detection. Show a ✌️ peace sign to trigger a selfie.')
try:
    while True:
        if IN_COLAB:
            frame = get_frame()  # BGR image
        else:
            ret, frame = cap.read()
            if not ret:
                print('Failed to grab frame')
                break
        last_frame = frame.copy()
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        h, w, _ = frame.shape
        display_frame = frame.copy()

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(display_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                if detect_peace_sign(hand_landmarks):
                    if not gesture_active:
                        gesture_active = True
                        gesture_start = time.time()
                        captured = False
                    # compute remaining countdown
                    elapsed = time.time() - gesture_start
                    remaining = max(0, int(DELAY_SECONDS - elapsed) + 1)
                    text = f'Taking Selfie in {max(0, round(DELAY_SECONDS - elapsed,1))}s'
                    cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2, cv2.LINE_AA)
                    # draw small countdown number
                    cv2.putText(display_frame, str(remaining), (w-80, 80), cv2.FONT_HERSHEY_DUPLEX, 3.0, (0,0,255), 6)
                    if elapsed >= DELAY_SECONDS and not captured:
                        # take selfie
                        filename = timestamp_filename('selfie', 'jpg')
                        full_path = os.path.join(SAVE_DIR, filename)
                        cv2.imwrite(full_path, last_frame)
                        print(f'Saved selfie to: {full_path}')
                        # apply filter and display result
                        filtered = apply_filter(last_frame, mode=FILTER)
                        out_name = os.path.join(SAVE_DIR, 'filtered_' + filename)
                        cv2.imwrite(out_name, filtered)
                        print(f'Saved filtered selfie to: {out_name}')
                        # optional sound: in Colab we can show a small JS beep (or just print)
                        try:
                            if IN_COLAB:
                                # play a short beep using JS audio (embedded)
                                eval_js(